#### Changing the directory

In [1]:
%pwd

'd:\\Office\\ODCRU\\02_During_Prep_Prog\\ODCRU-AI-Projects\\experiment'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\Office\\ODCRU\\02_During_Prep_Prog\\ODCRU-AI-Projects'

#### Use of ConfigBox

In [3]:
dicta = {"a":1,"b":2}
dicta["a"]

1

In [ ]:
# dicta.a   ### AttributeError as dict does not support attribute-style access 
            ### so we need to use a custom class or a library like box

AttributeError: 'dict' object has no attribute 'a'

In [5]:
from box import ConfigBox

dicta = ConfigBox(dicta)
dicta.a

1

#### Reading the YAML File

In [6]:
import re
def resolve_config(content: dict):
    '''
    This function resolves the placeholders in the configuration dictionary.
    Placeholders are in the format ${key1.key2} and will be replaced with the
    corresponding value from the config dictionary.
    Args:
        config (dict): The configuration dictionary with potential placeholders.
    Errors:
        Exception: Raises an exception if a placeholder cannot be resolved.
    Returns:
        dict: The configuration dictionary with placeholders resolved.
        
    '''
    pattern = re.compile(r"\$\{([^}^{]+)\}")

    def replacer(match):
        try:
            keys = match.group(1).split(".")
            value = content
            for k in keys:
                value = value[k]
            return str(value)
        except Exception as e:
            raise e

    def walk(d):
        for k, v in d.items():
            if isinstance(v, dict):
                walk(v)
            elif isinstance(v, str):
                d[k] = pattern.sub(replacer, v)

    walk(content)
    return content

In [9]:
import yaml

def read_yaml(path_to_yaml: str) -> ConfigBox:
    """Reads a yaml file and returns a ConfigBox object.
    Args:
        path_to_yaml (str): path like input
    Errors:
        exception: Empty file or any other exception while reading yaml file
    Returns:
        ConfigBox: A ConfigBox object containing the loaded YAML data
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            content = resolve_config(content)
            return ConfigBox(content)
    except Exception as e:
        raise e  
    
    

In [14]:
path_to_yaml = "config/config.yaml"
config = read_yaml(path_to_yaml)
config

ConfigBox({'artifacts_root': 'artifacts', 'YOLO8': {'model_name': 'yolov8n.pt', 'model_saved_path': '$(artifacts_root)/models/yolov8n.pt'}})

In [15]:

config.YOLO8.model_saved_path

'$(artifacts_root)/models/yolov8n.pt'